In [1]:
import sys
sys.path.append("..")

from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pickle
import yaml

from lib.notebooks import plot_groups_metrics
from imitative_agent import ImitativeAgent

current path: /mnt/c/Users/vpaul/OneDrive - CentraleSupelec/Inner_Speech/agent/imitative_agent


In [2]:
agents_path = glob("../out/imitative_agent1/*/")
agents_path.sort()

In [4]:
print(agents_path)

['../out/imitative_agent/bfa2f1a4bdf7d85496bc8c867342f96e-0/', '../out/imitative_agent/bfa2f1a4bdf7d85496bc8c867342f96e-1/', '../out/imitative_agent/bfa2f1a4bdf7d85496bc8c867342f96e-2/', '../out/imitative_agent/bfa2f1a4bdf7d85496bc8c867342f96e-3/', '../out/imitative_agent/bfa2f1a4bdf7d85496bc8c867342f96e-4/']


In [3]:
groups_metrics = {}

for agent_path in agents_path:
    agent = ImitativeAgent.reload(agent_path, load_nn=False)
    config = agent.config
    with open("%s/metrics.pickle" % agent_path, "rb") as f:
        metrics = pickle.load(f)
    
    if config['training']['jerk_loss_weight'] != 1:
        continue
    
    group_name = "\n".join((
        f"{','.join(config['dataset']['names'])}",
        f"synth_art={agent.synthesizer.config['dataset']['art_type']}",
        f"jerk_c={config['training']['jerk_loss_ceil']}",
        f"jerk_w={config['training']['jerk_loss_weight']}",
        f"bi={config['model']['inverse_model']['bidirectional']}",
    ))
    
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][agent_path] = metrics

{'dataset': {'batch_size': 8, 'datasplits_size': [64, 16, 20], 'names': ['pb2007'], 'num_workers': 6, 'shuffle_between_epochs': True, 'sound_type': 'cepstrum'}, 'model': {'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropout_p': 0.25, 'hidden_size': 32, 'num_layers': 2}}, 'synthesizer': {'name': 'ea587b76c95fecef01cfd16c7f5f289d-0/'}, 'training': {'jerk_loss_ceil': 0.014, 'jerk_loss_weight': 1, 'vel_loss_ceil': 0.014, 'vel_loss_weight': 1, 'learning_rate': 0.001, 'max_epochs': 500, 'patience': 25}}
{'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropout_p': 0.25, 'hidden_size': 32, 'num_layers': 2}}
{'direct_model': {'activation': 'relu', 'batch_norm': True, 'dropout_p': 0.25, 'hidden_layers': [256, 256, 256, 256]}, 'inverse_model': {'bidirectional': True, 'dropo

In [4]:
metrics_name = [
    "direct_model_estimation_error",
    "inverse_model_estimation_error",
    "inverse_model_repetition_error",
    "inverse_model_jerk",
]

def show_metrics(split_name="test"):
    plot_groups_metrics(groups_metrics, metrics_name, split_name)
display(ipw.interactive(show_metrics, split_name=["train", "validation", "test"]))

interactive(children=(Dropdown(description='split_name', index=2, options=('train', 'validation', 'test'), val…